In [ ]:
from matplotlib import pyplot as plt
plt.style.use("ggplot")
from scipy.integrate._ivp.ivp import OdeResult
import seaborn as sns
import numpy as np
import tqdm, os

from ars_247_2025_final_project.tram_model import TRAM_Model
from ars_247_2025_final_project.tools import gaussian_pulse, double_sigmoid_pulse

In [ ]:
OUTPUT_DIR = os.path.abspath(os.path.join("..", "Final_Project_PNG"))
print(f"Saving files to {OUTPUT_DIR}...")
HOUR = 3600
# transcriptional pulse
transcriptional_pulse = {
    'peak_time': 1.5 * HOUR, # 1hr peak
    'peak_value': 1,
    'width': 0.5 * HOUR
}

# phosphorylation pulse
phosphorylation_pulse = {
    'peak_time': 1.5 * HOUR // 4, # 20 minutes peak
    'peak_value': 1,
    'width': 0.5 * HOUR // 4 # 45 mins peak phosphorylation pulse
}

# optogenetic pulse
optogenetic_pulse = {
    'peak_time': 1.5 * HOUR // 10, # 2 minutes peak
    'peak_value': 1,
    'width': 0.5 * HOUR // 10 # 6 mins peak activity of pulse
}


## Figure 1

In [ ]:
system_params = {
    'PROM': 1,
    'V_A': 0.47, #um/s - mean for dynein / kinesin
    'R_M': 2.5, #um - mean for mammalian cells is 2.5; best results w/ 20
    'K_TEV': 0.015, #/s - physiological is ~0.15
    'K_TVMV': 0.015, #/s - physiological is ~0.15
    'K_TRAM_DEG': 0.00001, #/s - got interesting results with this being 0.01; physiological is ~0.00001
    'K_TF_DEG': 0.01 #/s - got interesting results with this being 0.1; physiological is ~0.01 w/ fast degron
}


In [ ]:
THIS_PULSE = optogenetic_pulse
models: dict[int, TRAM_Model] = {}
solutions: dict[int, OdeResult] = {}
T = 24 * HOUR
for i in tqdm.tqdm(range(1, 9)):
    models[i] = TRAM_Model(
        n_domains = i, 
        pulse = 'gaussian', pulse_params = THIS_PULSE, 
        sys_config = system_params
    )
    solutions[i] = models[i].solve_tram_ivp(0, T)

### Validating Input Pulse

In [ ]:
t = np.arange(0, 3 * HOUR)
pulse = [gaussian_pulse(y, **THIS_PULSE) for y in t]
plt.plot(t, pulse)

In [ ]:
# Area under pulse curve
THIS_N = 8
pulse_soln = models[THIS_N].solve_pulse()
plt.plot(pulse_soln['t'], pulse_soln['y'][0, :], color='k', label='Input')
ax = plt.gca()
ax.set_xticks(np.arange(0,3*HOUR,HOUR // 2))
ax.set_xticklabels(ax.get_xticks() / HOUR)

### Fig 1a
Demonstrating the output curve given physiological parameters

Starting with 4 TRAM domains

In [ ]:
trans_model_4_tram = TRAM_Model(
        n_domains = 4, 
        pulse = 'gaussian', pulse_params = transcriptional_pulse, 
        sys_config = system_params
)
soln_trans_model_4_tram = trans_model_4_tram.solve_tram_ivp(0, T)

In [ ]:
THIS_N = 4
pulse_soln = models[THIS_N].solve_pulse()
plt.axvline(THIS_PULSE['peak_time'], label='Pulse Peak (Input)', color='g', ls='--')
for i in list(range(THIS_N * 4))[::4]:
    this_alpha = i / 4 / THIS_N + 1 / THIS_N
    this_label = "Trafficking\nIntermediates" if i == THIS_N * (4 - 1) else None
    plt.plot(soln_trans_model_4_tram['t'], soln_trans_model_4_tram['y'][i, :], label=this_label, color='k', alpha=this_alpha)
plt.plot(soln_trans_model_4_tram['t'], soln_trans_model_4_tram['y'][-1, :], label='TF (Output)', color='magenta')
plt.ylabel("REU")
ax = plt.gca()
plt.xlim(0, 3.5 * HOUR)
ax.set_xticks(np.arange(0,3.5*HOUR,HOUR // 2))
ax.set_xticklabels(ax.get_xticks() / HOUR)
plt.xlabel("Time (hr)")
plt.figtext(
    x=0.03, y=0.86, s="a",
    horizontalalignment='left',
    verticalalignment='center',
    fontsize=12, fontweight='bold', style='italic'
)
plt.legend()
plt.savefig(os.path.join(OUTPUT_DIR, "Fig1a.png"), dpi=400)

### Fig 1b
Demonstrating the output curve given physiological parameters & optogenetic stimulation

Starting with 4 TRAM domains

In [ ]:
THIS_N = 4
pulse_soln = models[THIS_N].solve_pulse()
plt.axvline(THIS_PULSE['peak_time'], label='Pulse Peak (Input)', color='g', ls='--')
for i in list(range(THIS_N * 4))[::4]:
    this_alpha = i / 4 / THIS_N + 1 / THIS_N
    this_label = "Trafficking\nIntermediates" if i == THIS_N * (4 - 1) else None
    plt.plot(solutions[THIS_N]['t'], solutions[THIS_N]['y'][i, :], label=this_label, color='k', alpha=this_alpha)
plt.plot(solutions[THIS_N]['t'], solutions[THIS_N]['y'][-1, :], label='TF (Output)', color='magenta')
plt.ylabel("REU")
ax = plt.gca()
plt.xlim(0, 1 * HOUR)
ax.set_xticks(np.arange(0,1*HOUR,HOUR // 4))
ax.set_xticklabels(ax.get_xticks() / HOUR)
plt.xlabel("Time (hr)")
plt.figtext(
    x=0.03, y=0.86, s="b",
    horizontalalignment='left',
    verticalalignment='center',
    fontsize=12, fontweight='bold', style='italic'
)
plt.legend()
plt.savefig(os.path.join(OUTPUT_DIR, "Fig1b.png"), dpi=400)

### Fig 1c
Determining the relationship between TRAM domains and output peak time

In [ ]:
pulse_soln = models[THIS_N].solve_pulse()
plt.axvline(THIS_PULSE['peak_time'], label='Pulse Peak (Input)', color='g', ls='--')
for n_tram_domains, soln in solutions.items():
    this_alpha = n_tram_domains / len(models)
    plt.plot(soln['t'], soln['y'][-1,:], label=f"Output ({n_tram_domains}xTRAM)", color="m", alpha=this_alpha)
plt.ylabel("REU")
ax = plt.gca()
plt.xlim(0, 1 * HOUR)
ax.set_xticks(np.arange(0,1*HOUR,HOUR // 4))
ax.set_xticklabels(ax.get_xticks() / HOUR)
plt.xlabel("Time (hr)")
plt.figtext(
    x=0.03, y=0.86, s="c",
    horizontalalignment='left',
    verticalalignment='center',
    fontsize=12, fontweight='bold', style='italic'
)
plt.legend()
plt.savefig(os.path.join(OUTPUT_DIR, "Fig1c.png"), dpi=400)